Below are all the required packages for this notebook. Uncomment any packages you are missing and run the cell.

In [ ]:
#!pip install pandas
#!pip install numpy
#!pip install matplotlib
#!pip install seaborn
#!pip install pyproj
#!pip install folium
#!pip install scikit-learn
#!pip install imbalanced-learn
#!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
import re

# set display option to show all columns
pd.set_option('display.max_columns', None)

# load the dataset
df = pd.read_csv("traffic.csv", header=None)

# assign column names
column_names = [
    "Juhtumi nr", "Toimumisaeg", "Isikuid", "Hukkunuid", "Vigastatuid", 
    "Sõidukeid", "Aadress (PPA)", "Maja nr (PPA)", "Tänav (PPA)", 
    "Ristuv tänav (PPA)", "Maakond (PPA)", "Omavalitsus (PPA)", 
    "Asustus (PPA)", "Asula", "Liiklusõnnetuse liik [1]", 
    "Liiklusõnnetuse liik [3]", "Kergliikurijuhi osalusel", 
    "Jalakäija osalusel", "Kaassõitja osalusel", "Maastikusõiduki juhi osalusel",
    "Eaka (65+) mootorsõidukijuhi osalusel", "Bussijuhi osalusel", 
    "Veoautojuhi osalusel", "Ühissõidukijuhi osalusel", "Sõiduautojuhi osalusel",
    "Mootorratturi osalusel", "Mopeedijuhi osalusel", "Jalgratturi osalusel", 
    "Alaealise osalusel", "Turvavarustust mitte kasutanud isiku osalusel", 
    "Esmase juhiloa omaniku osalusel", "Mootorsõidukijuhi osalusel", 
    "Tüüpskeemi nr", "Tüüpskeem [2]", "Tee tüüp [1]", "Tee tüüp [2]", 
    "Tee element [1]", "Tee element [2]", "Tee objekt [2]", 
    "Kurvilisus", "Tee tasasus", "Tee seisund", "Teekate", 
    "Teekatte seisund [2]", "Sõiduradade arv", "Lubatud sõidukiirus (PPA)", 
    "Tee nr (PPA)", "Tee km (PPA)", "Ilmastik [1]", "Valgustus [1]", 
    "Valgustus [2]", "GPS X", "GPS Y"
]

df.columns = column_names

df["GPS X"] = pd.to_numeric(df["GPS X"], errors="coerce")
df["GPS Y"] = pd.to_numeric(df["GPS Y"], errors="coerce")

# function to check if time component is present
def time_present(date_str):
    if isinstance(date_str, str) and re.search(r'\d{1,2}[:]\d{2}', date_str):
        return True
    else:
        return False

# apply the function to check for time presence
df['Time_Present'] = df['Toimumisaeg'].apply(time_present)

# parse the 'Toimumisaeg' column
df['Parsed_Date'] = pd.to_datetime(df['Toimumisaeg'], dayfirst=True, errors='coerce')

# extract date features
df['Year'] = df['Parsed_Date'].dt.year
df['Month'] = df['Parsed_Date'].dt.month
df['DayOfWeek'] = df['Parsed_Date'].dt.dayofweek  # 0 = monday

# set 'Hour' to -1 if time is missing
df['Hour'] = df['Parsed_Date'].dt.hour
df.loc[~df['Time_Present'], 'Hour'] = -1

# handle missing time-based features
time_columns = ['Year', 'Month', 'DayOfWeek', 'Hour']
df[time_columns] = df[time_columns].fillna(-1).astype(int)

# clean numerical columns
numerical_columns = ['Isikuid', 'Hukkunuid', 'Vigastatuid', 'Sõidukeid', 'Sõiduradade arv']
df[numerical_columns] = df[numerical_columns].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

# create binary columns for fatalities and injuries
df['Fatal_Accident'] = (df['Hukkunuid'] > 0).astype(int)
df['Injury_Accident'] = (df['Vigastatuid'] > 0).astype(int)

# standardize categorical variables (strip whitespace and convert to lowercase)
categorical_columns = [
    'Maakond (PPA)', 'Ilmastik [1]', 'Tee tüüp [1]', 
    'Valgustus [1]', 'Tee seisund', 'Teekatte seisund [2]'
]
for col in categorical_columns:
    df[col] = df[col].astype(str).str.strip().str.lower()

# display the first few rows to verify
df.head()


# Task 2: Identifying High-Risk Locations and Conditions
In this section, we'll analyze the dataset to identify locations and conditions that are associated with higher accident counts and fatality rates.

#### Task 2a: High-Risk Locations
We'll identify the locations with the highest number of accidents and fatalities.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# filter out entries with missing 'Maakond (PPA)'
df_valid_locations = df[df['Maakond (PPA)'].notna()]

# accidents by county
accidents_by_county = df_valid_locations['Maakond (PPA)'].value_counts().reset_index()
accidents_by_county.columns = ['Maakond (PPA)', 'Accident_Count']

# plot accidents by county
plt.figure(figsize=(12, 6))
sns.barplot(data=accidents_by_county, x='Maakond (PPA)', y='Accident_Count')
plt.title('Accidents by County')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# fatal accidents by county
fatal_accidents_by_county = df_valid_locations[df_valid_locations['Fatal_Accident'] == 1]['Maakond (PPA)'].value_counts().reset_index()
fatal_accidents_by_county.columns = ['Maakond (PPA)', 'Fatal_Accident_Count']

# plot fatal accidents by county
plt.figure(figsize=(12, 6))
sns.barplot(data=fatal_accidents_by_county, x='Maakond (PPA)', y='Fatal_Accident_Count', color='red')
plt.title('Fatal Accidents by County')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


## Task 2b: High-Risk Conditions
#### We'll analyze environmental and road conditions to see which are most associated with accidents and fatalities.

In [ ]:
# define conditions to analyze
conditions = ['Ilmastik [1]', 'Tee tüüp [1]', 'Valgustus [1]', 'Tee seisund', 'Teekatte seisund [2]']

# calculate fatality risk for each condition
condition_risks = []

for condition in conditions:
    # filter out entries with missing condition data
    df_condition = df[df[condition].notna()]
    
    # create a crosstab of accidents and fatalities
    crosstab = pd.crosstab(df_condition[condition], df_condition['Fatal_Accident'])
    crosstab['Fatality_Risk'] = crosstab[1] / (crosstab[0] + crosstab[1])
    crosstab = crosstab.reset_index()
    crosstab['Condition'] = condition

    condition_risks.append(crosstab)

# combine all condition risks
condition_risks_df = pd.concat(condition_risks)

# visualize fatality risk by condition
for condition in conditions:
    subset = condition_risks_df[condition_risks_df['Condition'] == condition]
    subset = subset.sort_values('Fatality_Risk', ascending=False)
    
    plt.figure(figsize=(14, 8), constrained_layout=True)  # Larger figure with constrained layout
    sns.barplot(data=subset, x=condition, y='Fatality_Risk')
    plt.title(f'Fatality Risk by {condition}')
    plt.xticks(rotation=45, ha='right')  # Rotate labels for better fit
    plt.show()


Task 2c: Time-Based Analysis
We'll examine how accidents and fatalities vary by time of day, day of the week, and month.

In [ ]:
# filter out entries with invalid month
df_valid_time = df[df['Month'] != -1]

# accidents by hour
hourly_accidents = df_valid_time[df_valid_time['Hour'] != -1].groupby('Hour')['Juhtumi nr'].count().reset_index(name='Accident_Count')
hourly_fatal_accidents = df_valid_time[(df_valid_time['Fatal_Accident'] == 1) & (df_valid_time['Hour'] != -1)].groupby('Hour')['Juhtumi nr'].count().reset_index(name='Fatal_Accident_Count')

hourly_data = pd.merge(hourly_accidents, hourly_fatal_accidents, on='Hour', how='left').fillna(0)
hourly_data['Fatality_Risk'] = hourly_data['Fatal_Accident_Count'] / hourly_data['Accident_Count']

# plot accidents by hour
plt.figure(figsize=(12, 6))
sns.lineplot(data=hourly_data, x='Hour', y='Accident_Count', label='Total Accidents')
sns.lineplot(data=hourly_data, x='Hour', y='Fatal_Accident_Count', label='Fatal Accidents', color='red')
plt.title('Accidents by Hour')
plt.xticks(range(0, 24))
plt.tight_layout()
plt.show()

# plot fatality risk by hour
plt.figure(figsize=(12, 6))
sns.barplot(data=hourly_data, x='Hour', y='Fatality_Risk')
plt.title('Fatality Risk by Hour')
plt.xticks(range(0, 24))
plt.tight_layout()
plt.show()

# accidents by day of the week
day_labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_valid_time = df_valid_time[df_valid_time['DayOfWeek'] != -1]
df_valid_time['DayOfWeek_Label'] = df_valid_time['DayOfWeek'].apply(lambda x: day_labels[x] if x >= 0 and x < 7 else 'Unknown')

day_accidents = df_valid_time.groupby('DayOfWeek_Label')['Juhtumi nr'].count().reset_index(name='Accident_Count')
day_fatal_accidents = df_valid_time[df_valid_time['Fatal_Accident'] == 1].groupby('DayOfWeek_Label')['Juhtumi nr'].count().reset_index(name='Fatal_Accident_Count')

day_data = pd.merge(day_accidents, day_fatal_accidents, on='DayOfWeek_Label', how='left').fillna(0)
day_data['Fatality_Risk'] = day_data['Fatal_Accident_Count'] / day_data['Accident_Count']

# plot accidents by day of the week
plt.figure(figsize=(10, 6))
sns.barplot(data=day_data, x='DayOfWeek_Label', y='Accident_Count')
plt.title('Accidents by Day of the Week')
plt.tight_layout()
plt.show()

# plot fatality risk by day of the week
plt.figure(figsize=(10, 6))
sns.barplot(data=day_data, x='DayOfWeek_Label', y='Fatality_Risk')
plt.title('Fatality Risk by Day of the Week')
plt.tight_layout()
plt.show()

# accidents by month
monthly_accidents = df_valid_time.groupby('Month')['Juhtumi nr'].count().reset_index(name='Accident_Count')
monthly_fatal_accidents = df_valid_time[df_valid_time['Fatal_Accident'] == 1].groupby('Month')['Juhtumi nr'].count().reset_index(name='Fatal_Accident_Count')

monthly_data = pd.merge(monthly_accidents, monthly_fatal_accidents, on='Month', how='left').fillna(0)
monthly_data['Fatality_Risk'] = monthly_data['Fatal_Accident_Count'] / monthly_data['Accident_Count']

# exclude Month == -1 (already done above)

# plot accidents by month
plt.figure(figsize=(12, 6))
sns.barplot(data=monthly_data, x='Month', y='Accident_Count')
plt.title('Accidents by Month')
plt.tight_layout()
plt.show()

# plot fatality risk by month
plt.figure(figsize=(12, 6))
sns.barplot(data=monthly_data, x='Month', y='Fatality_Risk')
plt.title('Fatality Risk by Month')
plt.tight_layout()
plt.show()


## Task 2d: Driver Profiling
#### We'll analyze driver-related factors to identify profiles associated with higher accident involvement.

In [ ]:
# driver-related columns
driver_columns = [
    'Esmase juhiloa omaniku osalusel', 
    'Eaka (65+) mootorsõidukijuhi osalusel', 
    'Alaealise osalusel'
]

# ensure driver columns are numeric
for col in driver_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

# calculate fatality risk for each driver category
driver_risks = []

for col in driver_columns:
    total_accidents = df[col].sum()
    fatal_accidents = df[df['Fatal_Accident'] == 1][col].sum()
    fatality_risk = fatal_accidents / total_accidents if total_accidents > 0 else 0
    driver_risks.append({
        'Driver_Category': col,
        'Total_Accidents': total_accidents,
        'Fatal_Accidents': fatal_accidents,
        'Fatality_Risk': fatality_risk
    })

driver_risks_df = pd.DataFrame(driver_risks)

# visualize fatality risk by driver category
plt.figure(figsize=(10, 6))
sns.barplot(data=driver_risks_df, x='Driver_Category', y='Fatality_Risk')
plt.title('Fatality Risk by Driver Category')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


# Task 2e: High-Risk Streets and Spatial Analysis
In this section, we'll identify the streets with the highest number of accidents and fatalities.

# We'll then visualize these locations using GPS coordinates and create heatmap for Estonia.

Top 10 Streets with Highest Number of Accidents



In [ ]:
# Ensure 'Tänav (PPA)' is standardized
df['Tänav (PPA)'] = df['Tänav (PPA)'].astype(str).str.strip().str.lower()

# Count accidents by street
accidents_by_street = df['Tänav (PPA)'].value_counts().reset_index()
accidents_by_street.columns = ['Street', 'Accident_Count']

# Get top 10 streets
top10_accident_streets = accidents_by_street.iloc[1:11]
print("Top 10 Streets with Highest Number of Accidents:")
print(top10_accident_streets)


Top 10 Streets with Highest Number of Fatalities



In [ ]:
# Filter for fatal accidents
fatal_df = df[df['Fatal_Accident'] == 1]

# Count fatal accidents by street
fatalities_by_street = fatal_df['Tänav (PPA)'].value_counts().reset_index()
fatalities_by_street.columns = ['Street', 'Fatal_Accident_Count']

# Get top 10 streets
top10_fatal_streets = fatalities_by_street.iloc[1:11]
print("\nTop 10 Streets with Highest Number of Fatalities:")
print(top10_fatal_streets)


In [ ]:
fatal_df.head()

In [ ]:
# Ensure columns are numeric
fatal_df_gps = fatal_df[["GPS X", "GPS Y"]].copy()

# Drop rows with missing or invalid coordinates
fatal_df_gps = fatal_df_gps.dropna(subset=["GPS X", "GPS Y"])

# Check the data
print(fatal_df_gps.head())

In [ ]:
from pyproj import Transformer

# Initialize the transformer
transformer = Transformer.from_crs("epsg:3301", "epsg:4326", always_xy=True)

# Apply the transformation with swapped input (Y, X) for EPSG:3301
def convert_to_latlon(row):
    lon, lat = transformer.transform(row["GPS Y"], row["GPS X"])  # Swap the axes
    return pd.Series([lon, lat], index=["GPS X", "GPS Y"])

# Transform the coordinates
fatal_df_gps = fatal_df.apply(convert_to_latlon, axis=1)

# Round the results to 6 decimal places for clarity
fatal_df_gps["GPS X"] = fatal_df_gps["GPS X"].round(6)
fatal_df_gps["GPS Y"] = fatal_df_gps["GPS Y"].round(6)

# Check the converted data
print(fatal_df_gps.head())


Visualizing Streets on a Map
We'll use the folium library to visualize the top accident locations on a map.

In [ ]:
import folium
from folium.plugins import HeatMap

# Drop rows with NaN values, if any
fatal_df_gps = fatal_df_gps.dropna(subset=["GPS X", "GPS Y"])

# Center the map based on the mean coordinates
map_center_x = fatal_df_gps["GPS X"].mean()
map_center_y = fatal_df_gps["GPS Y"].mean()

# Create a Folium map
m = folium.Map(location=[map_center_y, map_center_x], zoom_start=7)

# Prepare data for the heatmap
heat_data = fatal_df_gps[["GPS Y", "GPS X"]].values.tolist()

# Add the HeatMap layer
HeatMap(heat_data).add_to(m)

# Display HeatMap
m

# Task 3: Data Preparation
In this section, we'll prepare the data for modeling. This includes handling missing values, encoding categorical variables, feature engineering, and splitting the data into training and testing sets

## Data Cleaning and Handling Missing Values
We'll first check for missing values and decide on strategies to handle them.

In [ ]:
# Improved date parsing function
def parse_date(date_str):
    date_formats = [
        '%d.%m.%Y %H:%M',
        '%d.%m.%Y',
        '%d/%m/%Y %H:%M',
        '%d/%m/%Y',
        '%d.%m.%y %H:%M',
        '%d.%m.%y',
        '%d/%m/%y %H:%M',
        '%d/%m/%y',
        '%Y-%m-%d %H:%M:%S',
        '%Y-%m-%d',
    ]
    for fmt in date_formats:
        try:
            return pd.to_datetime(date_str, format=fmt, dayfirst=True)
        except (ValueError, TypeError):
            continue
    return pd.NaT

# Apply the improved date parser
df['Parsed_Date'] = df['Toimumisaeg'].apply(parse_date)

# Drop rows with missing 'Parsed_Date'
df = df[df['Parsed_Date'].notna()].copy()

df.drop(columns=['Aadress (PPA)'], inplace=True)


## 3.1 Handling Missing Values

We'll clean numerical columns, handle missing numerical values, and standardize categorical columns. Additionally, we'll drop columns with excessive missing data.


In [ ]:
# Clean numerical columns
numerical_columns = [
    'Isikuid', 'Hukkunuid', 'Vigastatuid', 'Sõidukeid', 
    'Sõiduradade arv', 'Lubatud sõidukiirus (PPA)'
]
for col in numerical_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Fill missing numerical values
df['Sõiduradade arv'] = df['Sõiduradade arv'].fillna(df['Sõiduradade arv'].median())
df['Lubatud sõidukiirus (PPA)'] = df['Lubatud sõidukiirus (PPA)'].fillna(df['Lubatud sõidukiirus (PPA)'].median())

# For count-based columns, fill missing with 0
count_columns = ['Isikuid', 'Hukkunuid', 'Vigastatuid', 'Sõidukeid']
for col in count_columns:
    df[col] = df[col].fillna(0)

# Convert to integers
df[numerical_columns] = df[numerical_columns].astype(int)

# Standardize categorical variables
categorical_columns = [
    'Maakond (PPA)', 'Ilmastik [1]', 'Tee tüüp [1]', 
    'Valgustus [1]', 'Tee seisund', 'Teekatte seisund [2]'
]
for col in categorical_columns:
    df[col] = df[col].astype(str).str.strip().str.lower()
    df[col] = df[col].fillna('puudub')

# Drop columns with excessive missing data
df.drop(columns=['Asula',"Omavalitsus (PPA)",'Maja nr (PPA)', 'Ristuv tänav (PPA)', 'Asustus (PPA)', 'Tee nr (PPA)', 'Tee km (PPA)', "Tänav (PPA)"], inplace=True)


## 3.2 Encoding Categorical Variables

We'll encode binary categorical variables using label encoding and other categorical variables with one-hot encoding.


In [ ]:
# Label encoding for binary categorical variables
binary_columns = ['Jalakäija osalusel', 'Kaassõitja osalusel', 'Sõiduautojuhi osalusel','Esmase juhiloa omaniku osalusel', 'Eaka (65+) mootorsõidukijuhi osalusel', 'Alaealise osalusel']
for col in binary_columns:
    df[col] = df[col].map({'JAH': 1, 'jah': 1, 'EI': 0, 'ei': 0}).fillna(0).astype(int)

# One-hot encoding for other categorical variables
categorical_cols = [
    'Maakond (PPA)', 'Ilmastik [1]', 'Tee tüüp [1]', 'Valgustus [1]', 
    'Tee seisund', 'Teekatte seisund [2]'
]
df_encoded = pd.get_dummies(df, columns=categorical_cols)


## 3.3 Feature Engineering

We'll create a new feature for accident severity and time-based features.


In [ ]:

# Create time-based features
df_encoded['Is_Weekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)
df_encoded['Is_Night'] = df['Hour'].apply(lambda x: 1 if (x >= 22 or x <= 5) else 0)


## 3.4 Splitting the Data

We'll split the data into training and testing sets, ensuring no target leakage by removing 'Hukkunuid' and 'Vigastatuid' columns.


In [ ]:
from sklearn.model_selection import train_test_split


df_encoded.drop(columns=['Injury_Accident'], inplace=True, errors='ignore')

# Define the features and target variable
features = df_encoded.drop(['Juhtumi nr', 'Toimumisaeg', 'Parsed_Date', 'Time_Present', 'Fatal_Accident', 'Hukkunuid', 'Vigastatuid'], axis=1)
target = df_encoded['Fatal_Accident']

# Split the data<
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, stratify=target)


# Task 4: Modeling

In this task, we'll train models, evaluate them, and compare their performance.


In [ ]:
# Handling categorical columns for models
categorical_cols_to_encode = [
    'Liiklusõnnetuse liik [1]', 'Liiklusõnnetuse liik [3]', 'Tüüpskeemi nr', 
    'Tüüpskeem [2]', 'Tee tüüp [2]', 'Tee element [1]', 
    'Tee element [2]', 'Tee objekt [2]', 'Kurvilisus', 
    'Tee tasasus', 'Teekate', 'Valgustus [2]'
]

# Combine X_train and X_test for consistent encoding
X_combined = pd.concat([X_train, X_test], axis=0)

# Perform one-hot encoding
X_combined_encoded = pd.get_dummies(X_combined, columns=categorical_cols_to_encode)

# Split back into X_train and X_test
X_train_encoded = X_combined_encoded.iloc[:len(X_train), :].reset_index(drop=True)
X_test_encoded = X_combined_encoded.iloc[len(X_train):, :].reset_index(drop=True)

# Update X_train and X_test
X_train = X_train_encoded
X_test = X_test_encoded


## 4.1 Handling Class Imbalance

We'll use SMOTE to oversample the minority class in the training data.


In [ ]:
from imblearn.over_sampling import SMOTE

numeric_cols_to_convert = [
    'Kergliikurijuhi osalusel',
    'Maastikusõiduki juhi osalusel',
    'Bussijuhi osalusel',
    'Veoautojuhi osalusel',
    'Ühissõidukijuhi osalusel',
    'Mootorratturi osalusel',
    'Mopeedijuhi osalusel',
    'Jalgratturi osalusel',
    'Turvavarustust mitte kasutanud isiku osalusel',
    'Mootorsõidukijuhi osalusel',
    'GPS X',
    'GPS Y'
]

# Copy the original ones
X_train_original = X_train.copy()
y_train_original = y_train.copy()
X_test_original = X_test.copy()

# Convert to numeric, handling errors
for col in numeric_cols_to_convert:
    X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col], errors='coerce')
# Fill NaN values with appropriate values, e.g., 0
for col in numeric_cols_to_convert:
    X_train[col] = X_train[col].fillna(0)
    X_test[col] = X_test[col].fillna(0)

# Apply SMOTE
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)


## 4.2 Model Development

We'll train and evaluate Logistic Regression, Random Forest, and Gradient Boosting models.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# Logistic Regression
lr_model = LogisticRegression(max_iter=3000, random_state=42)
lr_model.fit(X_train_res, y_train_res)
y_pred_lr = lr_model.predict(X_test)
y_proba_lr = lr_model.predict_proba(X_test)[:, 1]

# Evaluate Logistic Regression
print("Logistic Regression:")
print(classification_report(y_test, y_pred_lr))
print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba_lr)}")


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest
rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=200,
    min_samples_split=2,
    min_samples_leaf=5,
    max_features='sqrt',
    bootstrap=True,
    criterion="entropy",
    class_weight='balanced',
    random_state=20
)

# Lets use the data before SMOTE, as it gets better results for random forests
rf_model.fit(X_train_original, y_train_original)
y_pred_rf = rf_model.predict(X_test_original)
y_proba_rf = rf_model.predict_proba(X_test_original)[:, 1]

# Evaluate Random Forest
print("Random Forest:")
print(classification_report(y_test, y_pred_rf))
print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba_rf)}")

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train_res, y_train_res)
y_pred_gb = gb_model.predict(X_test)
y_proba_gb = gb_model.predict_proba(X_test)[:, 1]

# Evaluate Gradient Boosting
print("Gradient Boosting:")
print(classification_report(y_test, y_pred_gb))
print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba_gb)}")


In [ ]:
from catboost import CatBoostClassifier

# CatBoost model
catboost_model = CatBoostClassifier(
    eval_metric='AUC',
    random_seed=42,
    verbose=100,
    early_stopping_rounds=200 
)

# Fit the model
cb_model = catboost_model
cb_model.fit(X_train_res, y_train_res)
y_pred_cb = cb_model.predict(X_test)
y_proba_cb = cb_model.predict_proba(X_test)[:, 1]

# Evaluate CatBoost
print("CatBoost:")
print(classification_report(y_test, y_pred_cb))
print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba_cb)}")

## 4.3 Model Comparison

We'll compare the models' performance based on ROC AUC scores.


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

# Function to plot the ROC curve for multiple models
def plot_multiple_roc_curves(models, y_test, probabilities, labels):
    plt.figure(figsize=(10, 8))
    for model, y_proba, label in zip(models, probabilities, labels):
        fpr, tpr, _ = roc_curve(y_test, y_proba)
        auc_score = roc_auc_score(y_test, y_proba)
        plt.plot(fpr, tpr, label=f'{label} (AUC = {auc_score:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves for All Models')
    plt.legend(loc='lower right')
    plt.grid()
    plt.show()

# Collect model probabilities and labels
probabilities = [y_proba_lr, y_proba_rf, y_proba_gb, y_proba_cb]
labels = ['Logistic Regression', 'Random Forest', 'Gradient Boosting', 'CatBoost']

# Plot all ROC curves
plot_multiple_roc_curves([lr_model, rf_model, gb_model, cb_model], y_test, probabilities, labels)


## Most critical features for determing fatality in the models

In [ ]:
# Get feature importance for Logistic Regression
lr_feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': lr_model.coef_[0]
})
lr_feature_importance['Abs_Coefficient'] = lr_feature_importance['Coefficient'].abs()
lr_feature_importance = lr_feature_importance.sort_values(by='Abs_Coefficient', ascending=False)

print("Top 10 Important Features for Logistic Regression:")

print(lr_feature_importance.head(10))

# Get feature importance for Random Forest
rf_feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
})
rf_feature_importance = rf_feature_importance.sort_values(by='Importance', ascending=False)

print("Top 10 Important Features for Random Forest:")
print(rf_feature_importance.head(10))


# Get feature importance for Gradient Boosting
gb_feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': gb_model.feature_importances_
})
gb_feature_importance = gb_feature_importance.sort_values(by='Importance', ascending=False)

print("Top 10 Important Features for Gradient Boosting:")
print(gb_feature_importance.head(10))

# Get feature importance for CatBoost
cb_feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': cb_model.feature_importances_
})
cb_feature_importance = cb_feature_importance.sort_values(by='Importance', ascending=False)

print("Top 10 Important Features for CatBoost:")
print(cb_feature_importance.head(10))



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plotting function for feature importance
def plot_feature_importance(df, model_name, top_n=10):
    plt.figure(figsize=(10, 6))
    sns.barplot(data=df.head(top_n), x='Importance', y='Feature')
    plt.title(f"Top {top_n} Features for {model_name}")
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.show()

# Plot for Random Forest
plot_feature_importance(rf_feature_importance, "Random Forest")

# Plot for Gradient Boosting
plot_feature_importance(gb_feature_importance, "Gradient Boosting")

# Plot for CatBoost
plot_feature_importance(cb_feature_importance, "CatBoost")

# For Logistic Regression
plot_feature_importance(lr_feature_importance.rename(columns={'Abs_Coefficient': 'Importance'}), "Logistic Regression")
